In [1]:
from common_imports import *

# JWST pipe version = 1.12.0
# STCAL version = 1.4.4


In [2]:
run_on_nsclean_data = True

if run_on_nsclean_data:
    stage1_dir = stage1_nsclean
    stage2_dir = stage2_nsclean
else:
    stage1_dir = stage1
    stage2_dir = stage2

In [4]:
def write_to_asn_sci(sci_files, bkg_files, asn_file, prod_name):
    """
    Write a .json association file that specifies a list of
    science and background exposures to be processed by Spec2.
    
    Arguments:
      * scifiles  :  List of on-source rate files to be processed.
      * bgfiles   :  List of background rate files to be processed.
      * asnfile   :  Output .json file path/name
      * prodname  :  Either 'Level2short' or 'Level2long', but this doesn't
                      really matter for much.
    
    Returns:
      * None
      
    Outputs:
      * Writes a .json association file to the location in `asnfile'
    """
    
    asn = asn_from_list(sci_files, rule=DMSLevel2bBase, product_name=prod_name)
    asn['asn_type'] = 'spec2'
    asn['asn_id']   = 'o001'
    asn['program']  = '1794'
    
    # Add background files to the association specified as exptype = background
    for bkg_file in bkg_files:
        for i in range(len(sci_files)):
            asn['products'][i]['members'].append({'expname': bkg_file, 
                                                  'exptype': 'background'})
    
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asn_file, 'w') as outfile:
        outfile.write(serialized)


def write_to_asn_bkg(bkg_files, asn_file, prod_name):
    """
    Write a .json association file that specifies a list of
    background exposures **ONLY** to be processed by Spec2.
    This can be useful to inspect the _cal.fits images of the
    background exposures to look for remaining artifacts.
    
    Arguments:
      * bgfiles   :  List of background rate files to be processed.
      * asnfile   :  Output .json file path/name
      * prodname  :  Either 'Level2short' or 'Level2long', but this doesn't
                      really matter for much.

    Returns:
      * None
      
    Outputs:
      * Writes a .json association file to the location in `asnfile'
    """
    
    asn = asn_from_list(bkg_files, rule=DMSLevel2bBase, product_name=prod_name)
    asn['asn_type'] = 'spec2'
    asn['asn_id']   = 'o001'
    asn['program']  = '1794'
    
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asn_file, 'w') as outfile:
        outfile.write(serialized)

# Similar to Stage 1, this function is where we specify how the Spec2 pipeline
# should be run.
def run_spec2(asn_file, out_dir, nocubes=False):
    """
    Run the Spec2 file using the given association file. The association
    file must contain the information needed - sci and bg exposures - to
    run the pipeline.
    
    Arguments:
      * asnfile   :  Input .json file path/name that specifies the sci+bg exposures
      * outdir    :  Directory for the stage2 output files
      * nocubes   :  By default Spec2 builds a datacube from each exposure,
                      but this is time-consuming and imo not really needed.
                      If nocubes=True, will not build exposures cubes.
    
    Returns:
      * None
      
    Outputs:
      * Writes _cal.fits files for each exposure to `outdir'. Also saves
          median-combined background images to `outdir'. Additionally if
          nocubes=False, writes a 3D data cube built from each exposure
          to `outdir'.
    """
    
    
    # This initial setup is just to make sure that we get the latest parameter reference files
    # pulled in for our files.  This is a temporary workaround to get around an issue with
    # how this pipeline calling method works.
    crds_config = Spec2Pipeline.get_config_from_reference(asn_file)
    spec2 = Spec2Pipeline.from_config_section(crds_config)
    spec2.output_dir = out_dir
    
    # Assign_wcs overrides
    #spec2.assign_wcs.override_distortion = 'myfile.asdf'
    #spec2.assign_wcs.override_regions = 'myfile.asdf'
    #spec2.assign_wcs.override_specwcs = 'myfile.asdf'
    #spec2.assign_wcs.override_wavelengthrange = 'myfile.asdf'

    # Flatfield overrides
    #spec2.flat.override_flat = 'myfile.fits'
        
    # Straylight overrides
    #spec2.straylight.override_mrsxartcorr = 'myfile.fits'
        
    # Fringe overrides
    #spec2.fringe.override_fringe = 'myfile.fits'
    
    # Photom overrides
    #spec2.photom.override_photom = 'myfile.fits'

    # Cubepar overrides
    #spec2.cube_build.override_cubepar = 'myfile.fits'
        
    # Extract1D overrides
    #spec2.extract1d.override_extract1d = 'myfile.asdf'
    #spec2.extract1d.override_apcorr = 'myfile.asdf'
        
    # Overrides for whether or not certain steps should be skipped
    # We do NOT want to skip background subtraction (although if this
    # is called on a bkg-only exposure it still gets processed correctly).
    spec2.assign_wcs.skip = False
    spec2.bkg_subtract.skip = False
    #spec2.flat_field.skip = False
    #spec2.srctype.skip = False
    spec2.straylight.skip = True # for MIRI
    spec2.fringe.skip = True # for MIRI
    #spec2.photom.skip = False
    #spec2.cube_build.skip = False
    #spec2.extract_1d.skip = False
    
    # Pipeline now does 2D image background subtraction in stage 2.
    # Save the combined background image it makes (it will save multiple copies
    # of the same background, one for each sci dither exposure...).
    spec2.bkg_subtract.save_combined_background = True
    
    # We can skip cube-building for the individual science frames if desired.
    # Useful in cases where we don't want need the cubes of each science frame.
    if nocubes:
        spec2.cube_build.skip = True
        spec2.extract_1d.skip = True
    
    # By default the ResidualFringeStep is turned off even in the new dev version
    # Cal files don't seem to show much obvious fringing at least in the long detector
    # so I will leave this off.
    # spec2.residual_fringe.skip = False
    
    # Some cube building options
    #spec2.cube_build.weighting='drizzle'
    spec2.cube_build.coord_system='ifualign' # If aligning cubes with IFU axes instead of sky
      
    spec2.save_results = True
    spec2(asn_file)


In [5]:
sci_files_nrs1 = sorted(glob.glob(stage1_dir+'/jw*1001*nrs1_rate*.fits'))
sci_files_nrs2 = sorted(glob.glob(stage1_dir+'/jw*1001*nrs2_rate*.fits'))

bkg_files_nrs1 = sorted(glob.glob(stage1_dir+'/jw*2001*nrs1_rate*.fits'))
bkg_files_nrs2 = sorted(glob.glob(stage1_dir+'/jw*2001*nrs2_rate*.fits'))

len(sci_files_nrs1), len(bkg_files_nrs1)

(20, 2)

In [6]:
# Medium grating doesn't project on NRS2 detector, so not processing those files
# otherwise, the pipeline will throw errors

asn_sci_nrs1 = os.path.join(stage2_dir, 'level2_sci_asn_nrs1.json')
# asn_sci_nrs2 = os.path.join(stage2, 'level2_sci_asn_nrs2.json')

asn_bkg_nrs1 = os.path.join(stage2_dir, 'level2_bkg_asn_nrs1.json')
# asn_bkg_nrs2 = os.path.join(stage2, 'level2_bkg_asn_nrs2.json')

write_to_asn_sci(sci_files_nrs1, bkg_files_nrs1, asn_sci_nrs1, 'Level2')
# write_to_asn_sci(sci_files_nrs2, bkg_files_nrs2, asn_sci_nrs2, 'Level2')

write_to_asn_bkg(bkg_files_nrs1, asn_bkg_nrs1, 'Level2')
# write_to_asn_bkg(bkg_files_nrs2, asn_bkg_nrs2, 'Level2')

run_spec2(asn_sci_nrs1, out_dir=stage2_dir, nocubes=True)
run_spec2(asn_bkg_nrs1, out_dir=stage2_dir, nocubes=True)

# run_spec2(asn_bkg_nrs2, out_dir=stage2, nocubes=True)
# run_spec2(asn_sci_nrs2, out_dir=stage2, nocubes=True)

2023-10-02 16:41:39,946 - stpipe - WARNING - /Users/ajshajib/miniforge3/envs/jwst_pipe/lib/python3.11/site-packages/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
2023-10-02 16:41:39,947 - stpipe - WARNING -   warnings.warn(err_str, UserWarning)
2023-10-02 16:41:39,947 - stpipe - WARNING - 
2023-10-02 16:41:40,230 - stpipe - WARNING - /Users/ajshajib/miniforge3/envs/jwst_pipe/lib/python3.11/site-packages/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
2023-10-02 16:41:40,230 - stpipe - WARNING -   warnings.warn(err_str, UserWarning)
2023-10-02 16:41:40,230 - stpipe - WARNING - 
2023-10-02 16:41:40,961 - CRDS - ERROR -  Error determining best reference for 'pars-pixelreplacestep'  =   Unknown reference type 'pars-pixelreplaceste